In [1]:
import sqlite3
import pandas as pd

In [2]:
# path = "../input/" 
# database = path + 'spotify.sqlite'
database = 'database.sqlite3'

conn = sqlite3.connect(database)
# Additional for solve error Could not decode to UTF-8 column (SQLITE)
conn.text_factory = lambda b: b.decode(errors = 'ignore')

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
print("Conection SuccessFull",conn)

Conection SuccessFull <sqlite3.Connection object at 0x0000014818707E20>


In [3]:
pd.read_sql("SELECT * FROM items", conn)

,id,name,gender,price,cost
0,1,rok,1,40,8
1,2,celana denim,0,65,29
2,3,celana denim,1,60,26
3,4,topi,2,15,3
4,5,blus,1,80,20
5,6,kaos kaki,2,10,2
6,7,sweater,1,34,2
7,8,celana pendek,1,32,11
8,9,celana pendek,0,22,9
9,10,kaos biru tua,2,20,9


In [4]:
pd.read_sql("SELECT * FROM sales_records", conn)

,id,user_id,item_id,purchased_at
0,1,204,28,2018-07-01
1,2,99,8,2018-07-01
2,3,97,17,2018-07-01
3,4,223,22,2018-07-01
4,5,130,19,2018-07-01
...,...,...,...,...
905,906,22,1,2018-07-31
906,907,100,24,2018-07-31
907,908,64,24,2018-07-31
908,909,113,26,2018-07-31


In [5]:
pd.read_sql("SELECT * FROM users", conn)

,id,name,gender,age
0,1,Giana,1,17
1,2,Diann,1,22
2,3,Antonella,1,22
3,4,Jodi,0,29
4,5,Fredra,1,54
...,...,...,...,...
320,321,Chandra,1,31
321,322,Dieter,0,15
322,323,Belia,1,35
323,324,Georgianna,1,48


In [6]:
# Membaca data dari SQL
df_items = pd.read_sql("SELECT * FROM items", conn)
df_sales = pd.read_sql("SELECT * FROM sales_records", conn)
df_users = pd.read_sql("SELECT * FROM users", conn)

# Menggabungkan DataFrame dengan inner join (Alternatif blend data yang bermasalah di looker)
sales_items = pd.merge(df_sales, df_items, left_on='item_id', right_on='id')
sales_users = pd.merge(df_sales, df_users, left_on='user_id', right_on='id')

# Menyimpan DataFrame ke dalam satu file Excel dengan beberapa sheet
with pd.ExcelWriter('data.xlsx') as writer:
    df_items.to_excel(writer, sheet_name='Items', index=False)
    df_sales.to_excel(writer, sheet_name='Sales', index=False)
    df_users.to_excel(writer, sheet_name='Users', index=False)
    sales_items.to_excel(writer, sheet_name='sales_items', index=False)
    sales_users.to_excel(writer, sheet_name='sales_users', index=False)

In [7]:
# # Membaca data dari SQL
# df_items = pd.read_sql("SELECT * FROM items", conn)
# df_sales = pd.read_sql("SELECT * FROM sales_records", conn)
# df_users = pd.read_sql("SELECT * FROM users", conn)

# # Menggabungkan DataFrame dengan inner join (Alternatif blend data yang bermasalah di looker)
# sales_items = pd.merge(df_sales, df_items, left_on='item_id', right_on='id')
# sales_users = pd.merge(df_sales, df_users, left_on='user_id', right_on='id')

# # Menggabungkan hasil dari sales_items dan sales_users
# sales_items_users = pd.merge(sales_items, df_users, left_on='user_id', right_on='id')

# # # Menyimpan DataFrame ke dalam satu file Excel dengan beberapa sheet
# # with pd.ExcelWriter('join.xlsx') as writer:
# #     df_items.to_excel(writer, sheet_name='Items', index=False)

In [ ]:
# Menggabungkan hasil dari sales_items dan sales_users
sales_items_users = pd.merge(sales_items, df_users, left_on='user_id', right_on='id')

In [9]:
print(sales_items_users.shape)
sales_items_users.head()

(910, 13)


,id_x,user_id,item_id,purchased_at,id_y,name_x,gender_x,price,cost,id,name_y,gender_y,age
0,1,204,28,2018-07-01,28,hoodie abu-abu,2,42,13,204,Lindi,1,33
1,421,204,28,2018-07-20,28,hoodie abu-abu,2,42,13,204,Lindi,1,33
2,587,204,17,2018-07-25,17,sepatu lari,0,75,28,204,Lindi,1,33
3,221,204,12,2018-07-14,12,kaos hitam,2,23,8,204,Lindi,1,33
4,124,204,25,2018-07-09,25,kaos polo hitam,2,37,16,204,Lindi,1,33


In [10]:
# last transaction date
sales_items_users["purchased_at"].max()

'2018-07-31'

In [21]:
import pandas as pd
from datetime import datetime

# Set a reference date for calculations
today_date = datetime(2018, 8, 1)

# Pastikan kolom 'TanggalTransaksi' dikonversi ke datetime
sales_items_users['purchased_at'] = pd.to_datetime(sales_items_users['purchased_at'])

# Group by customer ID and aggregate metrics: recency (days since last purchase), 
# frequency (number of purchases), monetary (total spend)
df_rfm = sales_items_users.groupby('user_id').agg(
        {'purchased_at': lambda date: (today_date - date.max()).days,
         'id_x': lambda num: num.nunique(),
         'price': lambda price: price.sum()})

In [23]:
df_rfm.columns = ['recency', 'frequency', "monetary"]  # Rename the aggregated columns
df_rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,283.0,6.657244,6.081524,1.0,2.0,4.0,10.0,28.0
frequency,283.0,3.215548,2.237536,1.0,2.0,3.0,4.0,14.0
monetary,283.0,165.296820,159.821255,10.0,56.5,110.0,218.5,1084.0


In [24]:
# Calculate recency score based on quantiles, assigning labels from 1 to 5 (1 being lowest recency)
df_rfm["recency_score"] = pd.qcut(df_rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
    
# Calculate frequency score based on rank quantiles, assigning labels from 1 to 5 (1 being lowest frequency)
df_rfm["frequency_score"] = pd.qcut(df_rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

df_rfm.head()

,recency,frequency,monetary,recency_score,frequency_score
user_id,,,,,
1,5,3,55,3,3
2,7,3,83,2,3
3,5,2,310,3,2
4,8,1,34,2,1
6,1,3,185,5,3


In [25]:
# Combine recency and frequency scores to create RFM segment
df_rfm['segment'] = df_rfm['recency_score'].astype(str) + df_rfm['frequency_score'].astype(str)
df_rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,segment
user_id,,,,,,
1,5,3,55,3,3,33
2,7,3,83,2,3,23
3,5,2,310,3,2,32
4,8,1,34,2,1,21
6,1,3,185,5,3,53


In [26]:
# Menyimpan DataFrame ke dalam satu file Excel
with pd.ExcelWriter('segmentation.xlsx') as writer:
    df_rfm.to_excel(writer, sheet_name='Items', index=False)